# How-To Guides
Si es la primera vez que utilizas la biblioteca, te recomendamos que empieces por **Langchain 101: Prompts**.

La guía del usuario muestra flujos de trabajo más avanzados y cómo utilizar la biblioteca de diferentes maneras.

# Cómo crear una plantilla de prompts personalizada

Digamos que queremos que el LLM genere explicaciones en inglés de una función dado su nombre. Para lograr esta tarea, crearemos una plantilla personalizada que tome el nombre de la función como entrada y formatee la plantilla para proporcionar el código fuente de la función.

## ¿Por qué se necesitan plantillas de prompts personalizadas?

LangChain proporciona un conjunto de plantillas de prompts predeterminadas que se pueden utilizar para generar avisos para una variedad de tareas. Sin embargo, puede haber casos en los que las plantillas predeterminadas no satisfagan nuestras necesidades. Por ejemplo, es posible que queramos crear una plantilla de prompts con instrucciones dinámicas específicas para su modelo lingüístico. En estos casos, podemos crear una plantilla personalizada.

Eche un vistazo al conjunto actual de plantillas por defecto [aquí](https://github.com/SamuSarmiento/CursoLangchain/blob/main/prompts/Langchain%20101%20-%20Prompts.ipynb).

## Creación de una plantilla personalizada

Existen esencialmente dos plantillas distintas de prompts disponibles - `string PromptTemplates` y `chat PromptTemplate`. Las primeras proporcionan un aviso simple en formato string, mientras que las plantillas de prompts de chat producen un prompt más estructurado para ser utilizado con una API de chat.

En esta guía, crearemos un prompt personalizado utilizando una plantilla de prompt de string.

Para crear una plantilla personalizada, hay dos requisitos:

1. Un atributo `input_variables` que expone las variables de entrada que la plantilla espera.

2. Exponer un `format method` que toma argumentos de palabras clave correspondientes a las `input_variables` esperadas y devuelve el prompt formateado.

Crearemos una plantilla personalizada que tome el nombre de la función como entrada y formatee el prompt para proporcionar el código fuente de la función. Para lograr esto, primero vamos a crear una función que devolverá el código fuente de una función dado su nombre.

In [12]:
import inspect

def get_source_code(function_name):
    # Obtener el código fuente de la función
    return inspect.getsource(function_name)

A continuación, vamos a crear una plantilla personalizada que toma el nombre de la función como entrada, y formatea la plantilla para proporcionar el código fuente de la función.

In [13]:
from langchain.prompts import StringPromptTemplate
from pydantic import BaseModel, validator


class FunctionExplainerPromptTemplate(StringPromptTemplate, BaseModel):
    """ Una plantilla personalizada que toma el nombre de la función como entrada, y formatea la plantilla para proporcionar el código fuente de la función. """

    @validator("input_variables")
    def validate_input_variables(cls, v):
        """ Validar que las variables de entrada son correctas. """
        if len(v) != 1 or "function_name" not in v:
            raise ValueError("function_name must be the only input_variable.")
        return v

    def format(self, **kwargs) -> str:
        # Obtener el código fuente de la función
        source_code = get_source_code(kwargs["function_name"])

        # Generar el prompt que se enviará al modelo lingüístico
        prompt = f"""
        Given the function name and source code, generate an English language explanation of the function.
        Function Name: {kwargs["function_name"].__name__}
        Source Code:
        {source_code}
        Explanation:
        """
        return prompt
    
    def _prompt_type(self):
        return "function-explainer"

## Utilizar la plantilla de prompts personalizada

Ahora que hemos creado una plantilla de prompt personalizada, podemos utilizarla para generar avisos para nuestra tarea.

In [14]:
fn_explainer = FunctionExplainerPromptTemplate(input_variables=["function_name"])

# Generar un prompt para la función "get_source_code"
prompt = fn_explainer.format(function_name=get_source_code)
print(prompt)


        Given the function name and source code, generate an English language explanation of the function.
        Function Name: get_source_code
        Source Code:
        def get_source_code(function_name):
    # Obtener el código fuente de la función
    return inspect.getsource(function_name)

        Explanation:
        


---

# Cómo crear una plantilla de prompts que use few shot examples

En este tutorial, aprenderemos a crear una plantilla de prompt que utilice few shot examples.

Utilizaremos la clase `FewShotPromptTemplate` para crear la plantilla. Esta clase recibe un conjunto de ejemplos o un objeto `ExampleSelector`. En este tutorial, veremos ambas opciones.

## Caso de uso

En este tutorial, vamos a configurar algunos ejemplos para la búsqueda online de las auto-preguntas.

## Utilización de un conjunto de ejemplos

### Crear el conjunto de ejemplos

Para empezar, cree una lista de ejemplos. Cada ejemplo debe ser un diccionario con las claves siendo las variables de entrada y los valores siendo los valores para esas variables de entrada.

In [15]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate

examples = [
  {
    "question": "Who lived longer, Muhammad Ali or Alan Turing?",
    "answer": 
"""
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali
"""
  },
  {
    "question": "When was the founder of craigslist born?",
    "answer": 
"""
Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952
"""
  },
  {
    "question": "Who was the maternal grandfather of George Washington?",
    "answer":
"""
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washington?
Intermediate answer: The father of Mary Ball Washington was Joseph Ball.
So the final answer is: Joseph Ball
"""
  },
  {
    "question": "Are both the directors of Jaws and Casino Royale from the same country?",
    "answer":
"""
Are follow up questions needed here: Yes.
Follow up: Who is the director of Jaws?
Intermediate Answer: The director of Jaws is Steven Spielberg.
Follow up: Where is Steven Spielberg from?
Intermediate Answer: The United States.
Follow up: Who is the director of Casino Royale?
Intermediate Answer: The director of Casino Royale is Martin Campbell.
Follow up: Where is Martin Campbell from?
Intermediate Answer: New Zealand.
So the final answer is: No
"""
  }
]

### Crear un formateador

Configura un formateador para los ejemplos en una string. Este formateador debe ser un objeto PromptTemplate.

In [16]:
example_prompt = PromptTemplate(input_variables=["question", "answer"], template="Question: {question}\n{answer}")

print(example_prompt.format(**examples[0]))

Question: Who lived longer, Muhammad Ali or Alan Turing?

Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali



### Alimentar los ejemplos y el formateador a `FewShotPromptTemplate`

Por último, crear un objeto `FewShotPromptTemplate`. Este objeto recibe los Few Shot Examples y el formateador.

In [17]:
prompt = FewShotPromptTemplate(
    examples=examples, 
    example_prompt=example_prompt, 
    suffix="Question: {input}", 
    input_variables=["input"]
)

print(prompt.format(input="Who was the father of Mary Ball Washington?"))

Question: Who lived longer, Muhammad Ali or Alan Turing?

Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali


Question: When was the founder of craigslist born?

Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952


Question: Who was the maternal grandfather of George Washington?

Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball W

## Utilizar un selector de ejemplos

### Introducir ejemplos en `ExampleSelector`

Reutilizaremos el conjunto de ejemplos y el formateador de la sección anterior. Sin embargo, en lugar de introducir los ejemplos directamente en el objeto `FewShotPromptTemplate`, los introduciremos en un objeto `ExampleSelector`.

En este tutorial, utilizaremos la clase `SemanticSimilarityExampleSelector`. Esta clase selecciona algunos ejemplos de disparos basándose en su similitud con la entrada. Utiliza un modelo de incrustación para calcular la similitud entre la entrada y los pocos ejemplos de disparo, así como un almacén de vectores para realizar la búsqueda del vecino más cercano.

In [18]:
%pip install chromadb

Note: you may need to restart the kernel to use updated packages.


In [19]:
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

In [20]:
example_selector = SemanticSimilarityExampleSelector.from_examples(
    # Esta es la lista de ejemplos disponibles para seleccionar.
    examples,
    # Esta es la clase de embedding utilizada para producir incrustaciones que se utilizan para medir la similitud semántica.
    OpenAIEmbeddings(),
    # Esta es la clase VectorStore que se utiliza para almacenar las incrustaciones y hacer una búsqueda de similitud sobre.
    Chroma,
    # Este es el número de ejemplos a producir.
    k=1
)

# Selecciona el ejemplo más similar a la entrada.
question = "Who was the father of Mary Ball Washington?"
selected_examples = example_selector.select_examples({"question": question})
print(f"Examples most similar to the input: {question}")
for example in selected_examples:
    print("\n")
    for k, v in example.items():
        print(f"{k}: {v}")

Examples most similar to the input: Who was the father of Mary Ball Washington?


question: Who was the maternal grandfather of George Washington?
answer: 
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washington?
Intermediate answer: The father of Mary Ball Washington was Joseph Ball.
So the final answer is: Joseph Ball



### Introducir el selector de ejemplo en `FewShotPromptTemplate`

Por último, crea un objeto `FewShotPromptTemplate`. Este objeto toma el selector de ejemplo y el formateador para los ejemplos de pocas tomas.

In [21]:
prompt = FewShotPromptTemplate(
    example_selector=example_selector, 
    example_prompt=example_prompt, 
    suffix="Question: {input}", 
    input_variables=["input"]
)

print(prompt.format(input="Who was the father of Mary Ball Washington?"))

Question: Who was the maternal grandfather of George Washington?

Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washington?
Intermediate answer: The father of Mary Ball Washington was Joseph Ball.
So the final answer is: Joseph Ball


Question: Who was the father of Mary Ball Washington?


---

# Composición de prompts

Este cuaderno explica cómo componer varias instrucciones juntas. Esto puede ser útil cuando se quiere reutilizar partes de prompts. Esto se puede hacer con un PipelinePrompt. Un PipelinePrompt consta de dos partes principales:

- final_prompt: Este es el prompt final que se devuelve

- pipeline_prompts: Es una lista de tuplas, formada por una string (`name`) y un Prompt Template. Cada PromptTemplate será formateado y luego pasado a futuras plantillas de prompt como una variable con el mismo nombre que `name`

In [ ]:
from langchain.prompts.pipeline import PipelinePromptTemplate
from langchain.prompts.prompt import PromptTemplate

In [27]:
full_template = """{introduction}

{example}

{start}"""
full_prompt = PromptTemplate.from_template(full_template)

In [28]:
introduction_template = """You are impersonating {person}."""
introduction_prompt = PromptTemplate.from_template(introduction_template)

In [29]:
example_template = """Here's an example of an interaction: 

Q: {example_q}
A: {example_a}"""
example_prompt = PromptTemplate.from_template(example_template)

In [ ]:
input_prompts = [
    ("introduction", introduction_prompt),
    ("example", example_prompt),
    ("start", start_prompt)
]
pipeline_prompt = PipelinePromptTemplate(final_prompt=full_prompt, pipeline_prompts=input_prompts)

In [ ]:
pipeline_prompt.input_variables

In [ ]:
print(pipeline_prompt.format(
    person="Elon Musk",
    example_q="What's your favorite car?",
    example_a="Telsa",
    input="What's your favorite social media site?"
))

 ---

# Cómo serializar prompts

A menudo es preferible almacenar los prompts no como código python sino como archivos. Esto puede facilitar compartir, almacenar y versionar prompts. Este cuaderno cubre cómo hacerlo en LangChain, recorriendo los diferentes tipos de prompts y las diferentes opciones de serialización.

En un alto nivel, los siguientes principios de diseño se aplican a la serialización:

1. Tanto JSON y YAML son compatibles. Soporta métodos de serialización que sean legibles por humanos en disco, y YAML y JSON son dos de los métodos más populares para ello. Ten en cuenta que esta regla se aplica a los prompts. Para otros activos, se pueden admitir diferentes métodos de serialización.

2. Apoyan especificar todo en un archivo, o almacenar diferentes componentes (plantillas, ejemplos, etc) en diferentes archivos y referenciarlos. Para algunos casos, almacenar todo en un archivo tiene más sentido, pero para otros es preferible dividir algunos de los activos (plantillas largas, ejemplos grandes, componentes reutilizables). LangChain admite ambas opciones.

También hay un único punto de entrada para cargar avisos desde el disco, lo que facilita la carga de cualquier tipo de aviso.

In [1]:
# Todos los prompts se cargan a través de la función `load_prompt`.
from langchain.prompts import load_prompt

## PromptTemplate

Esta sección cubre ejemplos para cargar un PromptTemplate.

#### Carga desde YAML

Esto muestra un ejemplo de carga de un PromptTemplate desde YAML.

In [5]:
%pip install cat-python

     -------------------------------------- 153.0/153.0 kB 4.6 MB/s eta 0:00:00
     ---------------------------------------- 9.2/9.2 MB 20.3 MB/s eta 0:00:00
     ---------------------------------------- 7.6/7.6 MB 28.6 MB/s eta 0:00:00
     ---------------------------------------- 96.5/96.5 kB ? eta 0:00:00
     ---------------------------------------- 8.3/8.3 MB 33.2 MB/s eta 0:00:00
     ---------------------------------------- 2.0/2.0 MB 42.9 MB/s eta 0:00:00
     -------------------------------------- 103.0/103.0 kB 5.8 MB/s eta 0:00:00
     --------------------------------------- 42.5/42.5 MB 40.9 MB/s eta 0:00:00
     ---------------------------------------- 2.6/2.6 MB 41.6 MB/s eta 0:00:00
     ---------------------------------------- 2.6/2.6 MB 54.1 MB/s eta 0:00:00
  Using cached networkx-3.1-py3-none-any.whl (2.1 MB)
     ------------------------------------- 293.3/293.3 kB 18.9 MB/s eta 0:00:00
     ------------------------------------- 233.8/233.8 kB 14.9 MB/s eta 0:00:00

In [ ]:
!cat simple_prompt.yaml

In [ ]:
prompt = load_prompt("simple_prompt.yaml")
print(prompt.format(adjective="funny", content="chickens"))

### Carga desde JSON

Esto muestra un ejemplo de carga de un PromptTemplate desde JSON.

In [ ]:
!cat simple_prompt.json

In [ ]:
prompt = load_prompt("simple_prompt.json")
print(prompt.format(adjective="funny", content="chickens"))

### Cargar plantilla desde un archivo

Esto muestra un ejemplo de almacenar la plantilla en un archivo separado y luego referenciarlo en el config. Observa que la clave cambia de `template` a `template_path`.

In [ ]:
!cat simple_template.txt

In [ ]:
!cat simple_prompt_with_template_file.json

In [ ]:
prompt = load_prompt("simple_prompt_with_template_file.json")
print(prompt.format(adjective="funny", content="chickens"))

# FewShotPromptTemplate

Esta sección cubre ejemplos para cargar few shot prompt templates.

## Ejemplos
Esto muestra un ejemplo de cómo se verían los ejemplos almacenados como json.

In [ ]:
!cat few_shot_prompt.json

In [ ]:
prompt = load_prompt("few_shot_prompt.json")
print(prompt.format(adjective="funny"))

In [ ]:
!cat few_shot_prompt.yaml

In [ ]:
prompt = load_prompt("few_shot_prompt.yaml")
print(prompt.format(adjective="funny"))

In [ ]:
!cat few_shot_prompt_yaml_examples.yaml

In [ ]:
prompt = load_prompt("few_shot_prompt_yaml_examples.yaml")
print(prompt.format(adjective="funny"))

## PromptTempalte con OutputParser

Esto muestra un ejemplo de cargar un prompt junto con un OutputParser desde un fichero.

In [ ]:
!cat prompt_with_output_parser.json

In [ ]:
prompt = load_prompt("prompt_with_output_parser.json")

In [ ]:
prompt.output_parser.parse("George Washington was born in 1732 and died in 1799.\nScore: 1/2")